In [1]:
from helper import *
#from Simulation.ForwardKinematics import ForwardKinematics
from Simulation.ForwardKinematics import ForwardKinematics

from Simulation.DifferentialKinematics import DifferentialKinematics
from Simulation.QP_DifferentialKinematics2 import QP_DifferentialKinematics

from neura_dual_quaternions import DualQuaternion, Quaternion
from Simulation.LinearInterpolator import LinearInterpolator

import roboticstoolbox as rtb
import spatialmath as sm
import spatialgeometry as sg
import numpy as np
from swift import Swift

import time
#np.set_printoptions(precision=3, suppress=True, linewidth=1200, threshold=np.inf)

In [4]:
dk = QP_DifferentialKinematics()
fk = ForwardKinematics()
q1 = np.array([0,  np.pi*0.5,  0,  0,  0, 0, 0.0])
ref = np.array([1, 1, 2, 0, 1, 2, 0.1, -2])

J = fk.getBodyJacobian8(q1)

A = dk.updateConstraintMatrix(dk.ConstraintMatrix, J, ref.reshape((8, 1))) 
print(A)

[[   3.000    3.000    3.000    3.000    3.000    3.000    3.000   -5.000]
 [   3.000    3.000    3.000    3.000    3.000    3.000    3.000   -5.000]
 [   3.000    3.000    3.000    3.000    3.000    3.000    3.000   -5.000]
 [   3.000    3.000    3.000    3.000    3.000    3.000    3.000   -5.000]
 [   3.000    3.000    3.000    3.000    3.000    3.000    3.000   -5.000]
 [   3.000    3.000    3.000    3.000    3.000    3.000    3.000   -5.000]
 [   3.000    3.000    3.000    3.000    3.000    3.000    3.000   -5.000]
 [   3.000    3.000    3.000    3.000    3.000    3.000    3.000   -5.000]
 [   1.000    0.000    0.000    0.000    0.000    0.000    0.000    0.000]
 [   0.000    1.000    0.000    0.000    0.000    0.000    0.000    0.000]
 [   0.000    0.000    1.000    0.000    0.000    0.000    0.000    0.000]
 [   0.000    0.000    0.000    1.000    0.000    0.000    0.000    0.000]
 [   0.000    0.000    0.000    0.000    1.000    0.000    0.000    0.000]
 [   0.000    0.000    0.

In [12]:
fk = ForwardKinematics()
q1 = np.array([0,  np.pi*0.5,  0,  0,  0, 0, 0.0])
#print(fk.getBodyJacobian(q1))
#print(fk.getGeometricJacobian(q1))

st = time.perf_counter()
fk.getBodyJacobian(q1)
et = time.perf_counter()
print(et-st)

st = time.perf_counter()
fk.getGeometricJacobian(q1)
et = time.perf_counter()
print(et-st)

0.0005277999953250401
0.0008279000030597672


In [3]:
x1 = DualQuaternion.fromQuatPos(Quaternion.fromAxisAngle(np.pi*0.5, np.array([0,1,0])), np.array([0.815,0,1.138]))
x2 = DualQuaternion.fromQuatPos(Quaternion.fromAxisAngle(np.pi*0.5, np.array([0,1,0])), np.array([0.815,0.2,1.138]))   

q0 = np.array([np.pi*0.5,np.pi*0.5,0,0,0,0,0])


J = fk.jacobian_body(q0)


# Compute the SVD
U, S, Vt = np.linalg.svd(J)

cartesian_direction = np.array([0,0,0, 0,0,0])  # Replace x, y, z with your direction components

# Normalize the direction vector
cartesian_direction = cartesian_direction / np.linalg.norm(cartesian_direction)

# Project the Cartesian direction onto the left singular vectors and multiply by singular values
manipulability_in_direction = np.dot(U*S,  cartesian_direction)
print(np.linalg.norm(manipulability_in_direction))
print(dk.dir_manipulability(q0, cartesian_direction))

AttributeError: 'ForwardKinematics' object has no attribute 'jacobian_body'

In [ ]:
pip install --upgrade neura-dual-quaternions

In [ ]:
q1 = np.array([-0.677,  0.208,  0.791,  1.547,  0.388, -0.141, -0.531])

print(dk.dir_manipulability_gradient3(q1))

print(dk.manipulability(q1))

In [ ]:
q1 = np.array([-0.677,  0.208,  0.791,  1.547,  0.388, -0.141, -0.531])
q2 = np.array([-0.687,  0.21,   0.804,  1.547,  0.393, -0.142, -0.54])
q3 = (q1 + q2)*0.5
v = [1, 0, 0, 0, 1, 0]
print(dk.dir_manipulability(q1, v))
print(dk.dir_manipulability(q3, v))
print(dk.dir_manipulability(q2, v))

print(dk.dir_manipulability_gradient(q1, v))
print(dk.dir_manipulability_gradient(q3, v))
print(dk.dir_manipulability_gradient(q2, v))

Jb = fk.jacobian_body(q1)
U, S, Vh = np.linalg.svd(Jb@Jb.T)
print(U)
print(S)

Jb = fk.jacobian_body(q3)
U, S, Vh = np.linalg.svd(Jb@Jb.T)
print(U)
print(S)

Jb = fk.jacobian_body(q2)
U, S, Vh = np.linalg.svd(Jb@Jb.T)
print(U)
print(S)

In [ ]:
# env = Swift()
# robot = rtb.models.URDF.Maira7M()

# env.launch(realtime=True, browser="notebook")
# env.add(robot)

# # end-effector axes
# ee_axes = sg.Axes(0.1)

# env.add(ee_axes)
# np.set_printoptions(suppress=True, precision=3)

# # Change the robot configuration to the ready position
# robot.q = np.array([0, 0.0, 0, np.pi / 2, 0.00, -0.0, 0])
# robot.qd = np.zeros(robot.n)




# # Step the sim to view the robot in this configuration
# env.step(0)

# # Specify our desired end-effector velocity
# ev = [0, 0.0, 0.0, 0, 0.0, 0]
# ev2 = [1, 0.0, 0.0, 0.0, 0.0, 0.0]
# # Specify our timestep
# dt = 0.005


# # Run the simulation for 4 seconds
# for i in range(1000):

#     # Work out the manipulator Jacobian, Hessian using the
#     # current robot configuration
#     J = fk.jacobian_body(robot.q)
#     grad = dk.manipulability_gradient(robot.q)
#     grad2 = dk.dir_manipulability_gradient(robot.q, ev2)
#     print(dk.manipulability(robot.q), dk.dir_manipulability(robot.q, ev2))
#     # Calculate δq, the change to qd
#     pinv = np.linalg.pinv(J)
#     v = pinv @ ev + 20*(np.eye(7) - pinv@J)@grad2

#     #robot.qd = qd - δq
#     robot.q += v*dt
    
#     # Update the ee axes
#     ee_axes.T = fk.forward_kinematics(robot.q).asTransformation()

#     # Step the simulator by dt seconds
#     env.step(dt)